In [1]:
%%bash
mkdir lab4
cd lab4
git clone https://github.com/sophiajwchoi/Lab4-Adding-Discovery-to-Chatbot.git

Cloning into 'Lab4-Adding-Discovery-to-Chatbot'...


#### At this point, you should see a folder (lab4) listed in the files directory in the left-sidebar of the JupyterLab environment. If this side menu is hidden, you can go to View>View Left-Sidebar. Select the lab4 and click on the  Lab4-Adding-Discovery-to-Chatbot subfolder. You will see a file called serverless.yml.

#### Replace the value of YOUR_USERNAME to "apikey", YOUR_DISCOVERY_PASSWORD to the API key from your Discovery service. Similarly, replace the values of ENVIRONMENT_ID and COLLECTION_ID with the values obtained earlier above. Replace the value of YOUR_DISCOVERY_URL with the url we obtained earlier. URL is location specific.

In [ ]:
%%bash
cd /resources/lab4/Lab4-Adding-Discovery-to-Chatbot
export npm_config_loglevel=silent
conda config --set notify_outdated_conda false
conda install nodejs -y
rm -f ~/.npmrc
npm install

#### Next you will login to your IBM Cloud account with your IBM Cloud email and password. Please ensure that you are logged in before running lines starting with ibmcloud. To login to your account, Create a new cell (as detailed earlier) and copy the below lines exactly in the cell you created. Replace YOUR_IBMCLOUD_EMAIL with your IBM Cloud email and YOUR_IBMCLOUD_PASSWORD with your password. Finally, run the cell, by pressing the play button (as mentioned earlier above). It may take a few minutes to finish. Please run the cell once. Please proceed to next step if you have successfully logged in without an error message.

In [ ]:
%%bash
ibmcloud config --check-version=false
ibmcloud login --no-region
YOUR_IBMCLOUD_EMAIL
YOUR_IBMCLOUD_PASSWORD

In [ ]:
%%bash
ibmcloud account orgs

#### Copy the lines below exactly in the new cell and replace REGION with region and Name_OF_ORG with the name of org you obtained above. If you are re-doing this step, please remove `ibmcloud account space-create 'lab4'` because you already created the space and it would give an error.

In [ ]:
%%bash
ibmcloud target --cf-api 'https://api.REGION.cf.cloud.ibm.com' -r REGION -o NAME_OF_ORG
ibmcloud account space-create 'lab4'

#### replace REGION (in line numbers 7 and 8) with region you obtained above

In [ ]:
%%bash
cd /resources/lab4/Lab4-Adding-Discovery-to-Chatbot
ibmcloud target -s lab4
ibmcloud plugin install cloud-functions -f
export npm_config_loglevel=silent
npm install -g serverless@1.51.0
ibmcloud fn --apihost REGION.functions.cloud.ibm.com
ibmcloud fn list --apihost REGION.functions.cloud.ibm.com
serverless deploy


In [ ]:
!ibmcloud fn list

## Another Part

In [ ]:
!pip install ibm_watson bs4

In [ ]:
import os
from glob import glob
from bs4 import BeautifulSoup
import IPython
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import SpeechToTextV1
from ibm_watson import AssistantV2
from ibm_watson import TextToSpeechV1

#### Copy your Speech to text API detalis and change YOUR_APIKEY, YOUR_ENDPOINT

In [ ]:
recognition_service = SpeechToTextV1(IAMAuthenticator('{YOUR_APIKEY}'))
recognition_service.set_service_url('{YOUR_ENDPOINT}')
SPEECH_EXTENSION = "*.webm"
SPEECH_AUDIOTYPE = "audio/webm"

def recognize_audio():
    while len(glob(SPEECH_EXTENSION)) == 0:
        pass
    filename = glob(SPEECH_EXTENSION)[0]
    audio_file = open(filename, "rb")
    os.popen("rm " + filename)
    result = recognition_service.recognize(audio=audio_file, content_type=SPEECH_AUDIOTYPE).get_result()
    return result["results"][0]["alternatives"][0]["transcript"]

#### Copy your Assistant API detalis and change YOUR_APIKEY, YOUR_ENDPOINT and YOUR_ASSISTANT_ID

In [ ]:
assistant = AssistantV2(version='2019-02-28', authenticator=IAMAuthenticator('{YOUR_APIKEY}'))
assistant.set_service_url('{YOUR_ENDPOINT}')
ASSISTANT_ID = "{YOUR_ASSISTANT_ID}"
session_id = assistant.create_session(assistant_id=ASSISTANT_ID).get_result()["session_id"]

def message_assistant(text):
    response = assistant.message(assistant_id=ASSISTANT_ID,
                                 session_id=session_id,
                                 input={'message_type': 'text', 'text': text}).get_result()
    return BeautifulSoup(response["output"]["generic"][0]["text"]).get_text()

#### Copy your Text to Speech API detalis and change YOUR_APIKEY, YOUR_ENDPOINT

In [ ]:
synthesis_service = TextToSpeechV1(IAMAuthenticator('{YOUR_APIKEY}'))
synthesis_service.set_service_url('{YOUR_ENDPOINT}')

def speak_text(text):
    with open('temp.wav', 'wb') as audio_file:
        response = synthesis_service.synthesize(text, accept='audio/wav', voice="en-US_AllisonV3Voice").get_result()
        audio_file.write(response.content)
    return IPython.display.Audio("temp.wav", autoplay=True)

In [ ]:
speak_text(message_assistant(recognize_audio()))